In [ ]:
import os
import sys
import re
import gc
import platform
import random
import matplotlib.pyplot as plt #графики
import plotly.express as px #графики
import seaborn as sns #графики


import numpy as np
import pandas as pd #пандас для загрузки датасета
from tqdm import tqdm # красивое отображение циклов

import torch #пайторч
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
import torchvision.transforms as transforms


from datasets import Dataset # датасет для хаггингфейс
import pandas as pd
from transformers import AutoImageProcessor, AutoModelForImageClassification, AutoModel, BeitForImageClassification # нужно, чтобы качать модели с hf

from sklearn.metrics import roc_auc_score, f1_score # рассчитывать метрики
from sklearn.model_selection import StratifiedKFold, train_test_split
import evaluate  # рассчитывать метрики

import timm # библиотека с нужными моделями
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
import glob
import cv2
from PIL import Image

import albumentations as A # аугментации
from albumentations.pytorch import ToTensorV2
from tqdm.contrib import tzip
import warnings
warnings.simplefilter('ignore')

2024-04-12 21:36:38.878744: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-12 21:36:38.903120: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-12 21:36:38.903146: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-12 21:36:38.903744: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-12 21:36:38.907997: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
model = timm.create_model('swinv2_base_window16_256', pretrained=True, num_classes=1)
config = resolve_data_config({}, model=model)
processor = create_transform(**config)
model

SwinTransformerV2(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (layers): Sequential(
    (0): SwinTransformerV2Stage(
      (downsample): Identity()
      (blocks): ModuleList(
        (0): SwinTransformerV2Block(
          (attn): WindowAttention(
            (cpb_mlp): Sequential(
              (0): Linear(in_features=2, out_features=512, bias=True)
              (1): ReLU(inplace=True)
              (2): Linear(in_features=512, out_features=4, bias=False)
            )
            (qkv): Linear(in_features=128, out_features=384, bias=False)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=128, out_features=128, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (drop_path1): 

In [ ]:
model.head.fc = model.head.flatten
model

SwinTransformerV2(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (layers): Sequential(
    (0): SwinTransformerV2Stage(
      (downsample): Identity()
      (blocks): ModuleList(
        (0): SwinTransformerV2Block(
          (attn): WindowAttention(
            (cpb_mlp): Sequential(
              (0): Linear(in_features=2, out_features=512, bias=True)
              (1): ReLU(inplace=True)
              (2): Linear(in_features=512, out_features=4, bias=False)
            )
            (qkv): Linear(in_features=128, out_features=384, bias=False)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=128, out_features=128, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (drop_path1): 

In [ ]:
model

SwinTransformerV2(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (layers): Sequential(
    (0): SwinTransformerV2Stage(
      (downsample): Identity()
      (blocks): ModuleList(
        (0): SwinTransformerV2Block(
          (attn): WindowAttention(
            (cpb_mlp): Sequential(
              (0): Linear(in_features=2, out_features=512, bias=True)
              (1): ReLU(inplace=True)
              (2): Linear(in_features=512, out_features=4, bias=False)
            )
            (qkv): Linear(in_features=128, out_features=384, bias=False)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=128, out_features=128, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (drop_path1): 

In [ ]:
df = pd.DataFrame(data={'path':[os.path.join('/home/misha/Documents/cp/train', id, os.listdir(f"train/{id}/")[0]) for id in all_id]})
df

,path
0,/home/misha/Documents/цп/train/19947009/206558...
1,/home/misha/Documents/цп/train/43803860/538980...
2,/home/misha/Documents/цп/train/9692761/1413983...
3,/home/misha/Documents/цп/train/9035567/5495004...
4,/home/misha/Documents/цп/train/10043161/693371...
...,...
16425,/home/misha/Documents/цп/train/24380034/529479...
16426,/home/misha/Documents/цп/train/10669754/786195...
16427,/home/misha/Documents/цп/train/21860275/285081...
16428,/home/misha/Documents/цп/train/9661966/6365461...


In [ ]:
def cos(nums1, nums2):
    return np.dot(nums1, nums2) / (np.linalg.norm(nums1) * np.linalg.norm(nums2))


id = '83137'
model.to('cuda')

all_id = os.listdir('train')


img = Image.open(f'train/{id}/'+os.listdir(f'train/{id}')[0]).convert('RGB')
img = processor(img).unsqueeze(0)
img = model(img.to('cuda'))[0].cpu().detach().numpy()

res = []
print(img)

for i in tqdm(all_id):
    for j in os.listdir(f'train/{i}'):
        img1 = Image.open(f'train/{i}/{j}').convert('RGB')
        img1 = processor(img1).unsqueeze(0)
        img1 = model(img1.to('cuda'))[0].cpu().detach().numpy()
        res.append([i, j, img1])



[-0.46584743 -0.27564043 -0.48388526 ...  0.93284065  0.6321738
  0.28462762]


100%|██████████| 16430/16430 [15:42<00:00, 17.43it/s] 


In [ ]:
import pickle

with open('emb1.pkl', 'wb') as f:
    pickle.dump(res, f)